<a href="https://colab.research.google.com/github/jimtodd92/CNN-MNIST/blob/master/IMDB_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMDB Datasets**
- Multiple datasets which describes IMDB ratings for movies, the movie details, cast and crew etc.

# **Task:**
1. Get the top 10 movies with minimum 50 votes and the ranking should be based on
(numVotes/averageNumberOfVotes)* averageRating
2. For the above 10 movies, list the persons who are most often credited and list the
different titles of the 10 movies.

# **Approach:**
Use Pyspark, windows, joins, filters, aggregations and other transformations

**Scalability:**
Yes, cluster can be vertically or horizonatally scaled.

# **Create Spark Session Object**

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.Builder().appName('IMDB Data Analysis').getOrCreate()

# **Load datasets from file**

In [42]:
df_name = spark.read.csv('./dataset/name.basics.tsv.gz', sep = "\t", header = True)
df_ratings = spark.read.csv('./dataset/title.ratings.tsv.gz', sep = "\t", header = True)
df_title_basics = spark.read.csv('./dataset/title.basics.tsv.gz', sep = "\t", header = True)

In [25]:
df_title_basics.show(10, truncate = False)

+---------+---------+-------------------------------------------+-------------------------------------------+-------+---------+-------+--------------+------------------------+
|tconst   |titleType|primaryTitle                               |originalTitle                              |isAdult|startYear|endYear|runtimeMinutes|genres                  |
+---------+---------+-------------------------------------------+-------------------------------------------+-------+---------+-------+--------------+------------------------+
|tt0000001|short    |Carmencita                                 |Carmencita                                 |0      |1894     |\N     |1             |Documentary,Short       |
|tt0000002|short    |Le clown et ses chiens                     |Le clown et ses chiens                     |0      |1892     |\N     |5             |Animation,Short         |
|tt0000003|short    |Pauvre Pierrot                             |Pauvre Pierrot                             |0      |189

In [58]:
from pyspark.sql.functions import *
from pyspark.sql import Window

# Filter movies with atleast 50 Num of votes.
df_ratings_50_votes = df_ratings.filter(df_ratings.numVotes >= 50)

# Populate averageNumOfVotes column.
df_ratings_with_avg = df_ratings_50_votes.withColumn('averageNumberOfVotes',lit(
  df_ratings.agg({'tconst': 'avg', 
               'averageRating': 'avg',
               'numVotes': 'avg'}).select(['avg(numVotes)']).first()[0]))

# Populate Score column.
# Score = (numVotes/averageNumberOfVotes)* averageRating.
df_ratings_with_score = df_ratings_with_avg.withColumn('Score', lit(
  round(
    (
        df_ratings_with_avg['numVotes']/df_ratings_with_avg['averageNumberOfVotes']
     )*df_ratings_with_avg['averageRating']
  )
)).orderBy(col('Score').desc())

# Window Specification for populate Rank based on Score.
tconst_window = Window.orderBy(desc("Score"))

# Filter first 10 movies with top 10 ranks.
df_ratings_with_rank = df_ratings_with_score.withColumn(
    'Rank',
     dense_rank().over(tconst_window)
     ).filter(col('Rank')<=10)

# **Top 10 movies based on calculated Score**

In [59]:
df_ratings_top_10 = df_ratings_with_rank.join(df_title_basics, df_title_basics.tconst==df_ratings_with_rank.tconst, 'left').select(
    df_title_basics.originalTitle,
    df_ratings_with_rank.tconst,
    df_ratings_with_rank.averageRating,
    df_ratings_with_rank.numVotes,
    round(df_ratings_with_rank.averageNumberOfVotes).alias('averageNumberOfVotes'),
    df_ratings_with_rank.Score,
    df_ratings_with_rank.Rank,
).orderBy('Rank')

In [60]:
df_ratings_top_10.show(truncate=False)

+------------------------+---------+-------------+--------+--------------------+-------+----+
|originalTitle           |tconst   |averageRating|numVotes|averageNumberOfVotes|Score  |Rank|
+------------------------+---------+-------------+--------+--------------------+-------+----+
|The Shawshank Redemption|tt0111161|9.3          |2545209 |999.0               |23704.0|1   |
|The Dark Knight         |tt0468569|9.0          |2495493 |999.0               |22491.0|2   |
|null                    |tt1375666|8.8          |2235341 |999.0               |19699.0|3   |
|Game of Thrones         |tt0944947|9.2          |1951975 |999.0               |17984.0|4   |
|Fight Club              |tt0137523|8.8          |2003393 |999.0               |17655.0|5   |
|Pulp Fiction            |tt0110912|8.9          |1957786 |999.0               |17449.0|6   |
|Forrest Gump            |tt0109830|8.8          |1963908 |999.0               |17307.0|7   |
|The Godfather           |tt0068646|9.2          |1750900 |9